In [ ]:
#librairies nécessaires au programme
import mido         # pour traiter fichier MIDI
import os           # pour parcourir le dossier lmd
import numpy as np  # pour créer et travailler avec les matrices
import sys          # pour traiter les erreurs

In [ ]:
def eliminate_midifile(file):
    """fonction pour vérifier si un fichier midi contient tous les instruments désirés ou pas
        la fonction s'occupe d'abord de vérifier les titres des 'tracks' (car c'est plus rapide).
        Puis elle vérifie les 20 messages de chaque 'track' dans le but de trouver le message 
        nommé 'program change' (qui correspond au numéro de programme de l'instrument (cf General MIDI)). 
        Elle retourne enfin une valeur booléenne correspondante."""
    has_drums = False
    has_piano = False
    has_guitar = False
    has_bass = False
    i_drums = []
    i_piano = []
    i_guitar = []
    i_bass = []
    
    piano_range = range(0,8)
    guitar_range = range(24,32)
    bass_range = range(32,40)
    
    #short conditions:
    for i,track in enumerate(file.tracks):
        names= track.name.split()
        for name in names:
            if name.lower()=='drums':
                has_drums=True
                i_drums.append(i)
                continue
            elif name.lower()=='piano':
                has_piano=True
                i_piano.append(i)
                continue
            elif name.lower()=='guitar':
                has_guitar=True
                i_guitar.append(i)
                continue
            elif name.lower()=='bass':
                has_bass=True
                i_bass.append(i)
                continue
    #testing if all tracks dont have the song name as their track name
    l=len(file.tracks)
    if len(i_drums)==l or len(i_piano)==l or len(i_guitar)==l or len(i_bass)==l:
        has_drums,has_piano,has_guitar,has_bass=False,False,False,False
        # if that's the case we can't be sure that track names correspond to the instruments,
        # so it's better to just scrap it and go along with the long conditions
        
    #long_conditions
    for track in file.tracks:
        for msg in track[0:20]: #first 20 msgs
            #condition 1 : 4/4 time signature on tempo data byte
            if msg.bytes()[:2]==[255,88]:
                if not(msg.bytes()[2:4]==[4,4]):
                    return False
            #condition 2 : drums 
            if has_drums==False:
                if msg.is_meta==False:
                    if msg.type()=='program_change':
                        if msg.dict()['channel']==9:
                            has_drums=True
            #condition 3 : piano
            if has_piano==False:
                if msg.is_meta==False:
                    if msg.type()=='program_change':
                        if msg.dict()['program'] in piano_range:
                            has_piano=True
            #condition 4 : guitar
            if has_guitar==False:
                if msg.is_meta==False:
                    if msg.type()=='program_change':
                        if msg.dict()['program'] in guitar_range:
                            has_guitar=True
            #condition 5 : bass
            if has_bass==False:
                 if msg.is_meta==False:
                    if msg.type()=='program_change':
                        if msg.dict()['program'] in bass_range:
                            has_bass=True
    #endloop
    if has_drums and has_piano and has_guitar and has_bass:
        return True
    else:
        return False
        

In [ ]:
def array_on_track(track,ticks):
    """code générant la matrice de données d'un instrument donné ('track')"""
    first_array = np.zeros(shape=(len(track),3))
    time = 0 #ticks
    for i,msg in enumerate(track):
        time+=msg.time
        if msg.type=='control_change' or msg.type=='program_change' or msg.type=='note_off':
            continue
        elif not msg.is_meta: # we only want note_on messages
            first_array[i,0]=1
            first_array[i,1]=msg.bytes()[1] #note_value 0..127
            # we scrap velocity
            first_array[i,2]=round(time*4/ticks) #quantized time in ticks
    # we have now an event-based list of notes
    # let's transform it into a time series with note value on the second dimension
    
    second_array = np.zeros(shape=(round(time*4/ticks),129))
    for note in first_array:
        if note[0]==1:
            second_array[int(note[2]),int(note[1])]=1
            second_array[int(note[2]),128]=1
    # in doing so, we skipped the data regarding velocity and note_off events
    # but, as this is just the input for the model and won't be used to create midifiles,
    # we can allow ourselves to simplify the data 
    return second_array

In [ ]:
def drum_array_on_track(track,ticks):
    """fonction qui crée les matrices de la batterie 
    avec les infos sur la vélocité et le décalage temporel (micro-timing) cette fois!
    """
    first_array = np.zeros(shape=(len(track),5))
    time = 0 #ticks
    
    #converting standard drum notation to indexes 0-8
    gm_to_ix ={
        35:0,36:0,
        37:1,38:1,39:1,40:1,
        42:2,44:2,
        46:3,
        41:4,43:4,
        45:5,47:5,
        48:6,50:6,
        51:7,53:7,59:7,
        49:8,52:8,55:8,57:8
    }
    #petit guide visuel:
    #kick,snare,HHc,HHo,tom3,tom2,tom1,ride,crash
    #0   ,1    ,2  ,3  ,4   ,5   ,6   ,7   ,8
    
    for i,msg in enumerate(track):
        time+=msg.time
        if msg.type=='control_change' or msg.type=='program_change' or msg.type=='note_off':
            continue
        elif not msg.is_meta:
            if msg.bytes()[1] in gm_to_ix.keys(): # to prevent key errors
                first_array[i,0]=1 #note or no note 
                first_array[i,1]=gm_to_ix[msg.bytes()[1]] #note_value 0-8
                first_array[i,2]=msg.bytes()[2]/127 #velocity : float 0..1
                first_array[i,3]=round(time*4/ticks) #quantized time in ticks
                first_array[i,4]=(time*4/ticks)-round(time*4/ticks) #offset in ticks (= time - quantized time)
    # remarque: "first array" n'est pas dense, il contient des trous correspondant aux 
    # évènements 'note_off' qui sont ignorés et aux autres messages épars inutiles
    
    #####
    #let's create the time series array:
    second_array=np.zeros((round(time*4/ticks),28))
    
    for note in first_array:
        if note[0]==1:
            # we do it this way to avoid needing multi-dimensional arrays.
            second_array[int(note[3]),int(note[1])]=1 # note
            second_array[int(note[3]),9+int(note[1])]=note[2] # vel
            second_array[int(note[3]),18+int(note[1])]=note[4] # time offset
            second_array[int(note[3]),27]=1 #tells us if a note is played at said timestep
            # this will probably be useful in the next step
            # when we'll try to find a window where all instruments are playing
    return second_array 

In [ ]:
def check_window(array1,array2,array3,array4,barsize):
    """fonction qui vérifie que toutes les instruments soient présents pour cette possibilité de matrice
        l'input est sont des arrays qui contiennent un 1 quand une note est jouée pour cet instrument, un 0 sinon

        plus concrètement, la fonction vérifie qu'il n'y ait pas de trous de plus de 2 mesures dans les matrices
    """
    #checking each 128x1-sized array
    for array in [array1,array2,array3,array4]:
        counter=0
        for val in array:
            if val==0: #note not found
                counter+=1
                if counter>2*barsize: 
                    # le seuil maximal sans notes est de 2 mesures, 
                    # après quoi cette fenètre est considérée comme inutile
                    # et la fonction exterieure passera à la prochaine fenètre 
                    return False
            else: #note found!!
                counter=0
            
    else:
        return True

In [ ]:
def create_arrays(file):
    """
    -> Fonction qui crée les matrices input et cible pour le modèle:
        on commence par compter les "tracks" et les grouper dans les catégories d'instuments qui nous intéressent,
        ensuite on parcourt chaque combinaison et le programme crée les matrices (ssi tous les instruments sont 
        présents dans la fenètre courante)
        le programme assemble les matrices des échantillons en les concaténant dans une matrice plus grande de rank+1 
        à la fin le programme retourne ces grandes matrices-là contenant chacune tous les échantillons d'un instrument
        pour ce morceau
    """

    ticks = file.ticks_per_beat
    
###j'utilise presque les mêmes catégories que MusicVAE: basse, batterie, et (à la place de "melody") guitare et piano
    nb_bass=0
    #let's split it in guitar and piano and have 1 instrument max per class
    nb_guitar=0
    nb_piano=0
    #nb_drums=1  # we don't care if there are more instruments for percussion,
    # because we only select those for drums (ctrl number : ca. 35-59), 
    # which correspond to the most important elements of the drumset (hihat,snare,etc.)
    
    piano_ix=[]
    guitar_ix=[]
    bass_ix=[]
    
    i_drums=0
    
    piano_range = range(0,8)
    guitar_range = range(24,32)
    bass_range = range(32,40)
    
    ##### filtering tracks:
    for i,track in enumerate(file.tracks):
        for msg in track[0:20]:
            if msg.is_meta == False:
                if msg.type=='program_change':
                    if msg.dict()['channel']==9:
                        i_drums=i
                    elif msg.dict()['program'] in piano_range:
                        nb_piano+=1
                        piano_ix.append(i)
                    elif msg.dict()['program'] in guitar_range:
                        nb_guitar+=1
                        guitar_ix.append(i)
                    elif msg.dict()['program'] in bass_range:
                        nb_bass+=1
                        bass_ix.append(i)
    #####
    created=False
    drum_array1 = drum_array_on_track(file.tracks[i_drums],ticks)
    for i_piano in piano_ix:
        for i_guitar in guitar_ix:
            for i_bass in bass_ix:
                #this loops over all possibilities for these instruments 
                piano_array1 = array_on_track(file.tracks[int(i_piano)],ticks)
                guitar_array1 = array_on_track(file.tracks[i_guitar],ticks)
                bass_array1 = array_on_track(file.tracks[i_bass],ticks)
                
                #######
                # now for the sliding window part
                max_length = min([
                    drum_array1.shape[0],
                    piano_array1.shape[0],
                    guitar_array1.shape[0],
                    bass_array1.shape[0]])
                #let's settle on 8 bars for our sliding window
                bar_size=16 #events per bar
                ws=128 # window_size = 8*16 # variable name was too long otherwise
                stride=32
                max_usable_length = max_length-ws
                
                for p in range(0,max_usable_length,stride):
                    #checking if there are all instruments in this selected window:
                    if check_window(
                        drum_array1[p:p+ws,27],
                        piano_array1[p:p+ws,128],
                        guitar_array1[p:p+ws,128],
                        bass_array1[p:p+ws,128],
                        bar_size
                    )==True:
                        
                        # adding to final arrays if already created
                        if created==False:
                            created=True
                            drum_final=drum_array1[p:p+ws,0:27].copy().reshape(1,ws,27)
                            piano_final=piano_array1[p:p+ws,0:128].copy().reshape(1,ws,128)
                            guitar_final=guitar_array1[p:p+ws,0:128].copy().reshape(1,ws,128)
                            bass_final=bass_array1[p:p+ws,0:128].copy().reshape(1,ws,128)
                        else:
                            drum_final=np.concatenate([
                                drum_final,
                                drum_array1[p:p+ws,0:27].copy().reshape(1,ws,27)],axis=0)
                            piano_final=np.concatenate([
                                piano_final,
                                piano_array1[p:p+ws,0:128].copy().reshape(1,ws,128)],axis=0)
                            guitar_final=np.concatenate([
                                guitar_final,
                                guitar_array1[p:p+ws,0:128].copy().reshape(1,ws,128)],axis=0)
                            bass_final=np.concatenate([
                                bass_final,
                                bass_array1[p:p+ws,0:128].copy().reshape(1,ws,128)],axis=0)
                    
    if created==True:
        return [drum_final,piano_final,guitar_final,bass_final]
    else:
        return False

In [ ]:
#ce code n'est nécessaire que pour continuer la création des données à l'endroit où le programme s'était arrêté auparavant
#   ça ouvre le fichier avec les abréviations des repertoires déjà traités parmi tous ceux du dossier de toutes les données
with open('dir_keywords.txt') as f:
      lines = f.readlines()
prev_dirs=[]
for line in lines:
    prev_dirs.append(line.strip('\n')) #nettoyage des abréviations 

In [ ]:
created = False
path='datasets/TM_dataset_enc9'
# si le programme s'arrête, il faut relancer avec cette nouvelle ligne pour ne pas écraser les données déjà traitées:
#   path='datasets/TM_dataset_enc9_partX'
 
i=0
try:
    for dirpath,dirnames,files in os.walk('datasets/lmd_matched'):
        if dirpath[21:24] in prev_dirs:
            continue
        elif dirpath[21:24] not in current_dirs:
                current_dirs.append(dirpath[21:24])
        print(f'\tDirectory : {dirpath}')
        with os.scandir(dirpath) as entries:
            for entry in entries:
                x ,extension = os.path.splitext(entry)
                if extension=='.mid':
                    #-----------------------------------------
                    if i%100==0:
                        print(f'file {i+1} out of 116,189')
                    if i%1000==0 and i!=0:
                        print('\t\t saving dataset on disk...')
                        if created:
                            np.savez_compressed(
                                path,
                                array1_drm=array_drum,
                                array2_pia=array_piano,
                                array3_gtr=array_gtr,
                                array4_bss=array_bass)
                        print('\t\t saving current_dirs on hist file...')
                        with open('dir_keywords.txt','a') as f: #append mode
                            for k in current_dirs:
                                f.write(k+'\n')
                        current_dirs.clear()
                    #pour ne pas perdre les progrès, je garde tous les chemins déjà parcourus
                    # sur un fichier .txt séparé; je sauve aussi localement tous les 1000 samples
                    # les datasets courants sur des fichiers compressés (np.savez_compressed())
                    #Étant donné que la fonction qui 'crawl' sur le dossier lmd-matched contenant toutes les chansons
                    # ne le fait pas de manière linéaire ou logique, je dois implémenter ces méthodes pour 
                    # m'y retrouver quand le programme crash (ce qui arrive quand même souvent)
                    #-----------------------------------------
                    #j'utilise beaucoup la fonctionnalité try... except de python ci-dessous.
                    # cela me permet de laisser tourner le programme longtemps et lorsqu'une erreur
                    # survient, je peux simplement l'ignorer et passer au prochain morceau du dataset
                    try:
                        file = mido.MidiFile(entry)
                        if not created:
                            created = True
                            try:
                                array_drum,array_piano,array_gtr,array_bass = create_arrays(file)
                            except KeyboardInterrupt:
                                raise KeyboardInterrupt
                            except:
                                created = False
                        else:
                            try:
                                array1,array2,array3,array4 = create_arrays(file)
                                array_drum=np.concatenate([array_drum,array1],axis=0)
                                array_piano=np.concatenate([array_piano,array2],axis=0)
                                array_gtr=np.concatenate([array_gtr,array3],axis=0)
                                array_bass=np.concatenate([array_bass,array4],axis=0)
                            except KeyboardInterrupt:
                                raise KeyboardInterrupt 
                                # j'ignore toutes les erreus sauf l'arrêt manuel forcé du programme
                            except:
                                pass
                    except KeyboardInterrupt:
                        
                        raise KeyboardInterrupt  
                    except:
                        print(f'\tfile {i+1} not valid, skipping..')
                    i+=1
except:
    print('\n',20*'-')
    print('Error type: KeyboardInterrupt')
    print('number of files read : ',i+1)
else:
    print('\n','end')
    print('number of files read : ',i+1)